In [30]:
import pandas as pd
from IPython.display import HTML, display
from os.path import join as pjoin
import csv
import pickle
import math
import heapq
from datetime import datetime


#m = pd.read_csv("Airbnb_Texas_Rentals.csv",encoding = 'utf8')

In [31]:
#we found words like '\\n' in the dataset so we cleaned it
#m = m.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\\n',  ' ', regex=True)
#m = m.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\\t',  ' ', regex=True)
#m = m.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\\r',  ' ', regex=True)

In [32]:
#create tsv files,we droped the first column that was a prroblem
#m = m.drop(['Unnamed: 0'], axis=1)

In [33]:
#we create the vocabulary of preprocessed documents,but we don't modify the documents because we''l use them in search engine
import nltk
import csv
import re
import os
import string
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+|\$')
ps = SnowballStemmer('english')

review_content_persist = {}
review_word_map_persist = {}
my_path = os.path.dirname(os.path.realpath('__file__'))


# Retrieving persisted information for review content and word map
with open(os.path.join(my_path, "indexes/review_content.pkl"), "rb") as review_content:
    review_content_persist = pickle.load(review_content)
    review_content.close()
    
with open(os.path.join(my_path, "indexes/review_word_map.pkl"), "rb") as review_word_map:
    review_word_map_persist = pickle.load(review_word_map)
    review_word_map.close()

if(len(review_word_map_persist.keys()) == 0):
    
    print("Indexes are being created")
    
    for i in range(18259):
        with open(os.path.join(my_path, 'docu_hw3/doc_' + str(i) + '.tsv'),encoding='utf8') as tsvfile:
             tsvreader = list(csv.reader(tsvfile, delimiter="\t"))
        l1 = tsvreader[0][4]
        l2 = tsvreader[0][7]
        l = l1+ ' ' +l2
        l = l.lower()
        l = tokenizer.tokenize(l)
        
        file_words = []
        
        for r in l :
            if not r in stop_words:
                sr = ps.stem(r)
                
                file_words.append(sr)
                
                if not  sr in review_word_map_persist:
                    review_word_map_persist[sr] = [i]
                else:
                    review_word_map_persist[sr]+=[i]
                    
                    
        review_content_persist[i] = ' '.join(file_words)
    
    with open(os.path.join(my_path, "indexes/review_content.pkl"), "wb") as review_content:
        pickle.dump(review_content_persist, review_content)
        review_content.close()
        
    with open(os.path.join(my_path, "indexes/review_word_map.pkl"), "wb") as review_word_map:
        pickle.dump(review_word_map_persist, review_word_map)
        review_word_map.close()
                

            

[nltk_data] Downloading package stopwords to C:\Users\Matteo
[nltk_data]     Cavalletti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
iindex_tf_idf_persist = {}

# Retriving precreated inverted indexes
with open(os.path.join(my_path, "indexes/iindex_tf_idf.pkl"), "rb") as iindex_tf_idf:
    iindex_tf_idf_persist = pickle.load(iindex_tf_idf)
    iindex_tf_idf.close()
    
if(len(iindex_tf_idf_persist.keys()) == 0):
    
    print("Inverted Indexes are being calculated")

    word_iindex = {}

    #Creating inverted index using tf-idf and consine similarity
    for word in review_word_map_persist:
        word_doc_list = review_word_map_persist[word]
        word_iindex[word] = []

        # Store indexes based on number of times a particular word is present in a given document
        for doc in word_doc_list:
            doc_content = review_content_persist[doc]
            word_iindex[word].append([doc, doc_content.split().count(word)])

    # Store indexes based on tf-idf
    docs_length = len(review_content_persist.keys())
    iindex_tf_idf_persist = word_iindex

    for key, word in iindex_tf_idf_persist.items():
        idf= math.log10( docs_length / len(word) )

        for elem in word:
            elem[1] = idf * elem[1]

    with open(os.path.join(my_path, "indexes/iindex_tf_idf.pkl"), "wb") as iindex_tf_idf:
        pickle.dump(iindex_tf_idf_persist, iindex_tf_idf)
        iindex_tf_idf.close()
        


In [35]:
# create dictionary of documents
# @TODO: Need to remove this dictionary if not used
dic_doc={}
for i in range(18259):
    dic_doc[r'docu_hw3\doc_'+str(i)+'.tsv']= i

In [36]:
word = input('Enter a search query: ')
    
def clean_input(w):
    w_list = []
    w = w.lower()
    w = tokenizer.tokenize(w)
    # Check if we need to do any other preprocessing to improve the efficiency of search results
    
    for r in w :
        if not r in stop_words:
            sr = ps.stem(r)
            if not  sr in w_list:
                w_list.append(sr)
    return w_list

word_list = clean_input(word)

print("Cleaned word: ", word_list)

list_doc_list = []

for w in word_list:
        doc_list = []
        
        if w in review_word_map_persist: 
            doc_list = review_word_map_persist[w]
            
        list_doc_list.append(doc_list)  
        
list_intersect = list_doc_list[0]

def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

        
for docList in list_doc_list:
    list_intersect = intersection(list_intersect, docList)
    
results = []

list_intersect = list(set(list_intersect))
        
for doc in list_intersect[:10]:
    with open(os.path.join(my_path, 'docu_hw3/doc_' + str(doc) + '.tsv'),encoding='utf8') as tsvfile:
         tsvreader = list(csv.reader(tsvfile, delimiter="\t"))
    
    title = tsvreader[0][7]        
    description = tsvreader[0][4]
    city = tsvreader[0][2]
    url = tsvreader[0][8]
    
    
    results.append([title, description, city, url])


    
if(len(results)):
    print('Found ' + str(len(list_intersect))  + ' matching reviews to your query')
    display(HTML('<table border="1"><tr><th>Title</th><th>Description</th><th>City</th><th>URL</th></tr><tr>{}</tr></table>'.format('</tr><tr>'.join('<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in results)
)))
else:
    display(HTML('<h1>No results found. Please try a different query</h1>'))
 

Enter a search query: welcome
Cleaned word:  ['welcom']
Found 1153 matching reviews to your query


In [37]:
dict_qcos = {}
dict_norm = {}


#print("Search started")

for doc in list_intersect[:100]:
    
    num = 0
    
    #print("Current document ID: " + str(doc))
    
    for word in word_list:
        w_index = iindex_tf_idf_persist[word]
        w_i_len = len(w_index)
        for i in range(w_i_len):
            if w_index[i][0] == doc: 
                num +=  w_index[i][1]
                
    dict_qcos[doc]=num

    
    #denominator
    norm = 0
    for word in iindex_tf_idf_persist.values():
        for i in range(len(word)):
            if word[i][0] == doc:
                norm +=  word[i][1]**2
    
    dict_norm[doc]=math.sqrt(norm)

#print("Numerator and Denominator calculated")
    
for doc,num in dict_qcos.items():
    if dict_norm[doc] != 0:
        dict_qcos[doc] = num/(math.sqrt(len(word_list))*dict_norm[doc])

h = []
results = []

#print("Cosine similarity done")

In [38]:
h = []
results = []

for doc in dict_qcos.keys():
    
    with open(os.path.join(my_path, 'docu_hw3/doc_' + str(doc) + '.tsv'),encoding='utf8') as tsvfile:
         tsvreader = list(csv.reader(tsvfile, delimiter="\t"))
    
    title = tsvreader[0][7]        
    description = tsvreader[0][4]
    city = tsvreader[0][2]
    url = tsvreader[0][8]
    
    
    heapq.heappush(h,(dict_qcos[doc], title, description, city, url))

#print("Applying Heap")    
heapq._heapify_max(h)

for i in range(10):
    
    results.append(list(heapq.heappop(h)))
    #print(heapq.heappop(h))

    heapq._heapify_max(h)

#print("Applying Heap")  

#print(results)
results_formatted = []


for i in results:
    
    first = round(i.pop(0), 4)
    i.append(first) 
    #first = round(float(results[i].pop(0)), 4)
    #results[i] = results[i].append(first)
    results_formatted.append(i)


if(len(results_formatted)):
    display(HTML('<table border="1"><tr><th>Title</th><th>Description</th><th>City</th><th>URL</th><th>Score</th></tr><tr>{}</tr></table>'.format('</tr><tr>'.join('<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in results_formatted)
)))
else:
    display(HTML('<h1>No results found. Please try a different query</h1>'))

Title,Description,City,URL,Score
Pets Welcome in Northwest Austin!,"Two guest bedrooms available in a residential home in a North Austin neighborhood. Close to Highways and just 15 miles from Downtown Austin (26 minute drive off-peak). Perfect for an affordable trip. One dog and two kitties live here, so must be comfortable with pets--yours are welcome as long as they're trained/well behaved. The neighborhood has a swimming pool which guests are welcome to access as well.",Austin,https://www.airbnb.com/rooms/18958178?location=Cedar%20Park%2C%20TX,0.8057
"Cozy Bungalow, walk from McKinney Square","My wife Georgia, our son Henry and I would be excited to welcome you into our home. You'll have a private room and bathroom and use of common areas. The McKinney square, with live music, food, shops and bars is walkable, and access to DFW is easy. We are excited to welcome people from all walks of life. People of color and LGBT folks are always welcome in our home.",McKinney,https://www.airbnb.com/rooms/2318159?location=Celina%2C%20TX,0.7
"Cozy Bungalow, walk from McKinney Square","My wife Georgia, our son Henry and I would be excited to welcome you into our home. You'll have a private room and bathroom and use of common areas. The McKinney square, with live music, food, shops and bars is walkable, and access to DFW is easy. We are excited to welcome people from all walks of life. People of color and LGBT folks are always welcome in our home.",McKinney,https://www.airbnb.com/rooms/2318159?location=Anna%2C%20TX,0.7
Getaway Gateway Apartment Home,This apartment is very nice for any family or group for a relaxing time or just a place to stay. Everyone is welcome. You are welcomed to use the pool and workout equipment at your leisure. This home also offers a full size garage for parking of two vehicles.,Katy,https://www.airbnb.com/rooms/17188329?location=Cinco%20Ranch%2C%20TX,0.5665
Artistic home in Lakewood close to downtown Dallas,"Our Lakewood place is close to downtown Dallas, as well as Uptown and Deep Ellum. You’ll love walking to restaurants and a grocery store--and beautiful White Rock Lake! We welcome all couples, solo adventurers, and business travelers. The master bedroom and bathroom are your retreat--and the shared spaces (kitchen, living room, screened in porch) are relaxing places that welcome you!",Dallas,https://www.airbnb.com/rooms/156130?location=Balch%20Springs%2C%20TX,0.4383
Comfy room and bed with full private bath,Welcome! This is a private room in a modest house located in a quiet subdivision in south College Station. Includes one bed in private bedroom and exclusive use of the hallway bathroom (with full tub/shower). Easy drive to campus. Must love dogs and cats. Diversity is welcome! *State and City Occupancy Taxes are included in rate* A fold-out bed is available for $10 fee (per stay).,College Station,https://www.airbnb.com/rooms/8789377?location=Bryan%2C%20TX,0.416
Luxury Private Bedroom & Bath in Las Colinas,"Welcome to your happy place - a newly renovated, well lit and furnished private bedroom with bath/shower over tub and accompanying closet area. Located in the heart of Las Colinas, a great neighborhood - quiet and surrounded by the lake. Guests from all backgrounds are most welcome here. Complimentary guest parking available. Good access to the highway - 2mins away.",Irving,https://www.airbnb.com/rooms/18180466?location=Coppell%2C%20TX,0.3667
Luxury Private Bedroom & Bath in Las Colinas,"Welcome to your happy place - a newly renovated, well lit and furnished private bedroom with bath/shower over tub and accompanying closet area. Located in the heart of Las Colinas, a great neighborhood - quiet and surrounded by the lake. Guests from all backgrounds are most welcome here. Complimentary guest parking available. Good access to the highway - 2mins away.",Irving,https://www.airbnb.com/rooms/18180466?location=Colleyville%2C%20TX,0.3667
Luxury Private Bedroom & Bath in Las Colinas,"Welcome to your happy place -

In [48]:
av_r_n_u = 0
bedr_c_u = 1000
d_list_u = datetime.strptime('January 1900', '%B %Y')
city_u = 'NA'
one_m_dif = datetime.strptime('February 2015', '%B %Y')- datetime.strptime('January 2015', '%B %Y')
two_m_dif = datetime.strptime('March 2015', '%B %Y')- datetime.strptime('January 2015', '%B %Y')
try:
    av_r_n_u = int(input('please, enter maximum price that you can pay(e.g. 10): '))
except:
    print('No price suggested')
    pass
try:
    bedr_c_u = int(input('please, enter the number of bedrooms that you need(e.g 5): '))
except:
    print('No indication about the needed number of bedrooms')
    pass
try:
    d_list_u = datetime.strptime(input('please, enter your needed period in this form,Month year(e.g. February 2015): '), '%B %Y')
except:
    print('No period selected')
    pass
try:
    city_u = input('please enter the city you want to reside(e.g. Austin): ')
except:
    print('No city was indicated')
    pass
list_ord =[]

for doc in list_intersect:
    with open(r'docu_hw3\doc_'+ str(doc) + '.tsv',encoding='utf8') as tsvfile:
         tsvreader = list(csv.reader(tsvfile, delimiter="\t"))
    title = tsvreader[0][7]        
    description = tsvreader[0][4]
    city = tsvreader[0][2]
    url = tsvreader[0][8]
    price = list(tsvreader[0][0])
    eff_price = ''
    for i in range(1,len(price)):
        eff_price += price[i]
    
    
    try:
        av_r_n = int(eff_price)
        bedr_c = int(tsvreader[0][1])
    except ValueError:
        av_r_n = 1000000
        bedr_c = 1
    
    d_list = datetime.strptime(tsvreader[0][3],'%B %Y')
    sum_values = 0
    if av_r_n >= 2*av_r_n_u :
               sum_values += 0          
    elif av_r_n < 2*av_r_n_u and av_r_n >= 1.75*av_r_n_u:
               sum_values += 0.25
    elif  av_r_n < 1.75*av_r_n_u and av_r_n >= 1.5*av_r_n_u:
               sum_values += 0.50  
    elif av_r_n < 1.5*av_r_n_u and av_r_n >= 1.25*av_r_n_u:
               sum_values += 0.75 
    elif av_r_n < 1.25*av_r_n_u and av_r_n >= av_r_n_u:
               sum_values += 1           
    elif av_r_n < av_r_n_u and av_r_n >=0.75*av_r_n_u:
               sum_values += 1.25
    elif av_r_n < 0.75*av_r_n_u and av_r_n >=0.5*av_r_n_u :
               sum_values += 1.5
    elif av_r_n < 0.5*av_r_n_u :
               sum_values += 2.
    
    
    if bedr_c >= bedr_c_u:
               sum_values += 1
    elif bedr_c == bedr_c_u-1:
               sum_values += 0.75
    elif bedr_c == bedr_c_u-2:
               sum_values += 0.5
    elif bedr_c <= bedr_c_u-3:
               sum_values += 0 #only to make index easier to understand , no practical effect
    
    if d_list == d_list_u:
               sum_values += 1
    elif d_list - d_list_u == one_m_dif or d_list - d_list_u == -one_m_dif :
               sum_values += 0.5
    elif d_list - d_list_u == two_m_dif or d_list - d_list_u == -two_m_dif :
               sum_values += 0.25
    else:
        sum_values += 0 #only to make index easier to understand , no practical effect
    
    if city_u == city :
        sum_values += 0.5
    
    score = round(sum_values/4.5,2)
    list_ord.append([title, description, city, url, str(score),score])
sorted(list_ord,key=lambda l:l[1]) 

results_formatted = []

for i in range(len(list_ord)):
    row = list_ord[i][:-1]
    results_formatted.append(row)

if(len(results_formatted)):
    display(HTML('<table border="1"><tr><th>Title</th><th>Description</th><th>City</th><th>URL</th><th>Score</th></tr><tr>{}</tr></table>'.format('</tr><tr>'.join('<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in results_formatted[:50])
)))
else:
    display(HTML('<h1>No results found. Please try a different query</h1>'))

please, enter maximum price that you can pay(e.g. 10): 10
please, enter the number of bedrooms that you need(e.g 5): 5
please, enter your needed period in this form,Month year(e.g. February 2015): February 2015
please enter the city you want to reside(e.g. Austin): Austin
